In [ ]:
import torch as tn
import torchtt as tntt
import matplotlib.pyplot as plt
import tt_iga
import numpy as np
import datetime
import scipy.sparse
import scipy.sparse.linalg
import pandas as pd

tn.set_default_dtype(tn.float64)

In [ ]:
Nt = 24                                                                
lz = 40e-3                                                             
Do = 72e-3                                                            
Di = 51e-3                                                            
hi = 13e-3                                                             
bli = 3e-3                                                             
Dc = 3.27640e-2                                                           
hc = 7.55176e-3                                                           
ri = 20e-3                                                           
ra = 18e-3                                                           
blc = hi-hc                                                           
rm = (Dc*Dc+hc*hc-ri*ri)/(Dc*np.sqrt(2)+hc*np.sqrt(2)-2*ri)                 
R = rm-ri
O = np.array([rm/np.sqrt(2),rm/np.sqrt(2)])
alpha1 = -np.pi*3/4       
alpha2 = np.math.asin((hc-rm/np.sqrt(2))/R)
alpha = np.abs(alpha2-alpha1)

A = np.array([[O[0] - ri/np.sqrt(2), O[1] - ri/np.sqrt(2)], [O[0] - Dc, O[1] - hc]])
b = np.array([[A[0,0]*ri/np.sqrt(2)+A[0,1]*ri/np.sqrt(2)],[A[1,0]*Dc+A[1,1]*hc]])
C = np.linalg.solve(A,b)

control_points1 = np.array([[Do,Do * np.tan(np.pi/8)],[Do/np.sqrt(2),Do/np.sqrt(2)],[Do/np.sqrt(2),Do/np.sqrt(2)],[rm/np.sqrt(2),rm/np.sqrt(2)],[ri/np.sqrt(2),ri/np.sqrt(2)]])
#knots2 = np.array([[Dc,hc],[Dc+blc,hi],[Di-bli,hi],[Di,hi-bli],[Di,0]])
control_points2 = np.array([[Di,hi-bli],[Di-bli,hi],[Di-bli,hi],[Dc+blc,hi],[Dc,hc]])
control_points3 = (control_points1+control_points2)/2
control_points3[-1,:] = C.flatten()
control_points = np.concatenate((control_points1[:,None,:],control_points3[:,None,:],control_points2[:,None,:]),1)
weights = np.ones((5,3))
weights[1,-1] = np.sin((np.pi-alpha)/2)


basis1 = tt_iga.bspline.BSplineBasis(np.array(np.linspace(0,1,4)),2)
basis2 = tt_iga.bspline.BSplineBasis(np.array([0,1]),2)

geom = tt_iga.PatchNURBS([basis1, basis2],[], [tntt.TT(control_points[:,:,0]), tntt.TT(control_points[:,:,1])], tntt.TT(weights))


In [ ]:
y1,y2 = tn.linspace(0,1,100), tn.linspace(0,1,100)

Xs = geom([y1.flatten(),y2.flatten()])
plt.figure()

plt.scatter(Xs[0].numpy().flatten(),Xs[1].numpy().flatten(),s=0.25)

xg1, Mg1 = basis1.interpolating_points()
xg2, Mg2 = basis2.interpolating_points()

xs = geom([xg1.flatten(),xg2.flatten()])
f = xs[0].numpy()**2+xs[1].numpy()**2
dofs = np.linalg.solve(np.einsum('ij,kl->ikjl',Mg1,Mg2).reshape([15,-1]),f.reshape([-1,1])).reshape([5,3])
# dofs = np.zeros((5,3))
# dofs[2,1] = 1
# dofs[1,1] = 1
z = tn.outer(tn.sin(10*y1),tn.cos(10*y2))
z = np.einsum('ij,im,jn->mn',dofs,basis1(y1),basis2(y2))
plt.figure()
plt.contour(Xs[0].numpy(),Xs[1].numpy(),z,levels=20)


